In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

C:\Users\dougl\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dougl\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dougl\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dougl\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework

In [3]:
base = pd.read_csv('iec6100429.csv')

In [4]:
x = base.drop('Forma de onda', axis=1) # atributos previsores
y = base.iloc[:,-1] # pegando a ultima coluna para saber qual a classe

In [5]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories = 'auto')
y.shape, x.shape

((27,), (27, 19001))

In [6]:
y = y.values.reshape(-1,1)
y.shape

(27, 1)

In [7]:
y = onehot.fit_transform(y).toarray()

In [30]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size = 0.3)

In [31]:
x.shape, x_teste.shape, y.shape, y_teste.shape

((27, 19001), (9, 19001), (27, 3), (9, 3))

In [32]:
neuronios_entrada = x.shape[1]
neuronios_entrada

19001

In [33]:
neuronios_oculta1 = int((neuronios_entrada + y.shape[1])/2)
neuronios_oculta1

9502

In [34]:
neuronios_saida = y.shape[1]
neuronios_saida

3

In [12]:
# 100 -> 51 -> 3

In [35]:
w = {'oculta1': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta1])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta1, neuronios_saida]))}

In [36]:
b = {'oculta1': tf.Variable(tf.random_normal([neuronios_oculta1])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}

In [37]:
xph = tf.placeholder('float', [None, neuronios_entrada])
yph = tf.placeholder('float', [None, neuronios_saida])

In [58]:
def mlp(x, w, bias):
    camada_oculta1 = tf.nn.relu(tf.add(tf.matmul(x, w['oculta1']), b['oculta1']))
    camada_saida = tf.add(tf.matmul(camada_oculta1, w['saida']), b['saida'])
    return camada_saida

In [59]:
modelo = mlp(xph, w, b)

In [60]:
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(erro)

In [61]:
batch_size = 18
batch_total = int(len(x_treinamento) / batch_size)
batch_total, x_treinamento.shape

(1, (18, 19001))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(500):
        erro_medio = 0.0
        batch_total = int(len(x_treinamento) / batch_size)
        x_batches = np.array_split(x_treinamento, batch_total)
        y_batches = np.array_split(y_treinamento, batch_total)
        for i in range(batch_total):
            x_batch, y_batch = x_batches[i], y_batches[i]
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: x_batch, yph: y_batch})
            erro_medio += custo / batch_total
        if epoca % 30 == 0:
            print('Época: ' + str(epoca+1) + ' erro: '+ str(erro_medio))
    w_final, b_final = sess.run([w,b])
    
    print('fim do treinamento')
    
    previsoes = tf.nn.softmax(mlp(xph, w_final, b_final))
    previsoes_corretas = tf.equal(tf.argmax(previsoes, 1), tf.argmax(yph, 1))
    taxa_acerto = tf.reduce_mean(tf.cast(previsoes_corretas, tf.float32))
    print(sess.run(taxa_acerto, feed_dict = {xph: x_teste, yph: y_teste}))

Época: 1 erro: 10227.64453125
Época: 31 erro: 44575.22265625
Época: 61 erro: 11121.6279296875
Época: 91 erro: 13366.6123046875
Época: 121 erro: 36162.2265625
Época: 151 erro: 15040.41015625
Época: 181 erro: 17087.583984375
Época: 211 erro: 4916.072265625
Época: 241 erro: 8973.96484375
Época: 271 erro: 7803.87158203125
Época: 301 erro: 11754.2763671875
Época: 331 erro: 2981.1015625
Época: 361 erro: 11609.3935546875
Época: 391 erro: 1735.111328125
Época: 421 erro: 10495.2822265625
Época: 451 erro: 4108.41259765625
Época: 481 erro: 11490.55859375
fim do treinamento
